In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import re

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 10

In [1]:
# Load income data
df_county_crime = pd.read_csv('income_crime_with_percentages.csv')
df_county_crime

NameError: name 'pd' is not defined

In [1]:
# Load county income data
df_county_income = pd.read_csv('income_county.csv')
df_county_income

NameError: name 'pd' is not defined

In [ ]:
# Extract county and state from url
df_county_income['countystate'] = df_income['place'].str.extract("place/(.*)/")
df_county_income['countystate']

In [ ]:
# Extract state
df_county_income['state'] = df_income['countystate'].str.extract("(.*)/")
df_county_income['state']


In [ ]:
# Extract city
df_county_income['county'] = df_income['countystate'].str.extract("/(.*)")
df_county_income['couny']

In [ ]:
# Remove place columns
df_county_income = df_income.drop(['place', 'citystate'], axis=1)
df_county_income

In [ ]:
# Reorder columns
df_county_income = df_county_income[['Unnamed: 0', 'city', 'state', 'percentile', 'income', 'percent']]
df_county_income

In [ ]:
# Remove zeroes in income and percent columns
df_county_income['income'] = df_income['income'].replace({'0 ':''}, regex = True)
df_county_income['percent'] = df_income['percent'].replace({'0 ':''}, regex = True)
df_county_income

In [ ]:
# Convert income to int
pd.options.mode.chained_assignment = None 
for i in range(0, len(df_income)):
    incomeConvert = df_county_income['income'][i]
    incomeConvert = incomeConvert[:(incomeConvert.find("."))]
    incomeConvert = incomeConvert.replace(',', "")
    incomeConvert = re.sub(r"^\s+", "", incomeConvert, flags=re.UNICODE)
    
    incomeConvert = int(float(incomeConvert))
    df_income['income'][i] = incomeConvert
    #print(incomeConvert)    

In [ ]:
# Drop 'percent' column
# Make percentile columns 20th, 40th, Median, 60th, 80th, 95th
df_percentile = df_county_income.pivot_table(index=["state","city"], columns='percentile', values=['income'], aggfunc='first')
df_percentile.columns = df_percentile.columns.droplevel()
df_percentile = df_percentile.reset_index()
df_percentile.columns=df_percentile.columns.tolist()
df_percentile = df_percentile[['city','state','20th','40th','Median','60th','80th','95th']]
df_percentile

In [ ]:
# Merge income percentile dataframe with crime dataframe
df = pd.merge(df_percentile, df_county_crime, on=['city','state'])
df = df.drop(['Unnamed: 0'], axis=1)
df

In [ ]:
# Create inequality column: 95th minus 20th
inequality = [0]*len(df)
for i in range(0,len(df)):
    inequality[i] = df['95th'][i] - df['20th'][i]

df['income inequality'] = inequality  
df

In [ ]:
df.to_csv('income_county_crime.csv')